# Machine Learning Exercise

En la industria de la publicidad online, se llama impresión a cuando se muestra un anuncio a un usuario, y click, a cuando el usuario clickea el mismo, y en los casos en los que se hace click, ambos eventos son guardados compartiendo un identificador correspondiente a la transacción.

A menudo se considera una tarea importante la capacidad de las empresas para predecir que usuarios van a hacer click en un anuncio antes de mostrarlo, ya que puede haber más de un anuncio disponible y mostrarlo incurre un costo.

Se cuenta con una base de datos con el siguiente esquema:

![alt text](./scheme_imps_clicks.png)


Aquí ([descargar](https://drive.google.com/uc?id=1fmhC3Wjvp_n7pMcKKL8ZQPXZng5QW8bb&export=download)) se cuenta con una base de datos sqlite3 con datos de ejemplo, para trabajar.

La misma cuenta con una muestra de unos pocos días de datos, en la que las variables categóricas han sido hasheadas para mantener los nombres anónimos, esto puede considerarse irrelevante, las mismas corresponden a datos como el país donde ocurrió la impresión o a que cliente y proveedor corresponde.




1. Utilizar la base de datos para entrenar un modelo para estimar si una impresión va a ser clickeada.
Nota: se considera que una impresión fue clickeada, si hay un click con el mismo id cuyo timestamp es lo sumo 2 horas mayor al de la impresión
Se deben utilizar los primeros días de datos como datos de entrenamiento y el último día como test para evaluar su modelo.
2. Describa el modelo, algoritmo y metodología utilizada.
3. Presente brevemente los resultados obtenidos. Como los evaluó? Por qué?
4. Bonus: Cambiaría algo a su solución si la base de datos fuese 10 veces más grande que la provista? Que?

In [50]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

In [2]:
cnx = sqlite3.connect('imps_clicks.db.2')

In [3]:
impressions = pd.read_sql_query("SELECT * FROM impressions", cnx)
# impressions.to_csv('impressions.csv')
# impressions = pd.read_csv('impressions.csv')

In [4]:
clicks = pd.read_sql_query("SELECT * FROM clicks", cnx)

In [5]:
print(impressions.shape)
print(clicks.shape)
print(impressions.shape[0] == len(impressions['id'].unique()))
print(clicks.shape[0] == len(clicks['id'].unique()))

(3634110, 26)
(145809, 3)
True
True


In [6]:
impressions.head()

,id,time,r_bidfloor,r_age,r_height,r_width,categorical_0,categorical_1,categorical_2,categorical_3,...,categorical_10,categorical_11,categorical_12,categorical_13,categorical_14,categorical_15,categorical_16,categorical_17,categorical_18,categorical_19
0,292168921339466749,1.525719e+09,0.090,NaN,NaN,NaN,0f92dddc,0b6f5a3c,ac9c5a5e,324df420,...,e2dd1f89,af748bc4,539438fe,81b73927,0f92dddc,e5a8600b,be9d46cb,4fb33344,afe7b405,f46f184b
1,301026010107952934,1.525719e+09,0.569,NaN,250.0,300.0,0f92dddc,8ed5aa01,acdf3b27,5df13856,...,e2dd1f89,af748bc4,539438fe,60ccf532,0f92dddc,a6175c97,4efbfa20,0f92dddc,afe7b405,28322c68
2,1593468802636566196,1.525719e+09,0.219,NaN,250.0,300.0,48454cb2,75a8d373,bf95698f,81d47546,...,83634957,c78aa531,539438fe,d43589af,b6ecc8b2,a6175c97,4efbfa20,0f92dddc,afe7b405,ec0691fc
3,851314676801512203,1.525719e+09,0.930,NaN,NaN,NaN,0f92dddc,b6866c56,47c41a85,85ac804d,...,6ff49f13,c78aa531,539438fe,f8f36dd0,0f92dddc,0774a6df,be9d46cb,1f24f24a,afe7b405,6b5b9e8f
4,91561499709735199,1.525721e+09,6.730,NaN,480.0,320.0,0c436334,8ed5aa01,5419c88d,f9e3361a,...,e2dd1f89,c78aa531,539438fe,81b73927,b6ecc8b2,9beb5470,be9d46cb,1181e38f,afe7b405,dd867f0a


In [7]:
clicks.head()

,id,time,categorical_0
0,65183818012886860,1.525722e+09,0af10f2c
1,2113542931232678124,1.525723e+09,fea08526
2,1082639249992126326,1.525722e+09,e8e0e1a5
3,1635761239458084446,1.525722e+09,21c51bb5
4,1877180330329664957,1.525723e+09,b4b7efe0


### Ordenamos los datos por timestamp

In [8]:
impressions = impressions.sort_values(by=['time'])
clicks = clicks.sort_values(by=['time'])

In [9]:
impressions['time'] = impressions['time'].apply(lambda t: pd.Timestamp(t))
clicks['time'] = clicks['time'].apply(lambda t: pd.Timestamp(t))

### Renombramos columnas y hacemos merge

In [10]:
impressions = impressions.rename(columns={'time': 'imp_time'})
clicks = clicks.rename(columns={'time': 'click_time'})
imp_click = impressions.merge(clicks, on='id', how='inner')

In [11]:
imp_click.head()

,id,imp_time,r_bidfloor,r_age,r_height,r_width,categorical_0_x,categorical_1,categorical_2,categorical_3,...,categorical_12,categorical_13,categorical_14,categorical_15,categorical_16,categorical_17,categorical_18,categorical_19,click_time,categorical_0_y
0,1388967106491182997,1970-01-01 00:00:01.525637403,0.07,NaN,90.0,728.0,0f92dddc,463cfef1,a4e20423,99c882da,...,2366e25f,0f92dddc,0f92dddc,8e28f4a3,be9d46cb,358f2411,afe7b405,06a90c7d,1970-01-01 00:00:01.525651721,a880baf8
1,11802553372880683,1970-01-01 00:00:01.525637602,0.43,NaN,50.0,320.0,0f92dddc,8ed5aa01,acdf3b27,b1e20f71,...,539438fe,81b73927,b6ecc8b2,72dc346c,be9d46cb,67e4e242,afe7b405,3bfe91dd,1970-01-01 00:00:01.525651628,21295cac
2,1886751662102221837,1970-01-01 00:00:01.525637773,0.43,NaN,50.0,320.0,0f92dddc,8ed5aa01,acdf3b27,b1e20f71,...,539438fe,81b73927,0f92dddc,72dc346c,be9d46cb,67e4e242,afe7b405,17ff64ac,1970-01-01 00:00:01.525652088,be2e9927
3,1570875775553557833,1970-01-01 00:00:01.525638101,0.99,NaN,480.0,320.0,0f92dddc,8ed5aa01,acdf3b27,5df13856,...,539438fe,a1170f96,0f92dddc,25cb891d,be9d46cb,986327ca,afe7b405,35b6662d,1970-01-01 00:00:01.525652010,536bbdf1
4,57575448066589591,1970-01-01 00:00:01.525638128,0.85,NaN,480.0,320.0,0f92dddc,8ed5aa01,acdf3b27,1e52fa9f,...,539438fe,f2149314,0f92dddc,72dc346c,be9d46cb,9b612e9e,afe7b405,28322c68,1970-01-01 00:00:01.525652377,05d1eec4


### Filtramos los datos correspondientes impresiones clickeadas (tiempo de click <= 2hs del tiempo de impresion)

In [12]:
two_hours = pd.Timedelta(value=2, unit='h')

In [13]:
print(imp_click['click_time'][0])
print(imp_click['imp_time'][0])
print(imp_click['click_time'][0] - imp_click['imp_time'][0] <= two_hours)

1970-01-01 00:00:01.525651721
1970-01-01 00:00:01.525637403
True


In [14]:
imp_click = imp_click[imp_click['click_time'] - imp_click['imp_time'] <= two_hours]

### Obtenemos los id de las impresiones que fueron clickeadas

In [15]:
clicks_id = imp_click['id'].unique()
clicks_id

array([1388967106491182997,   11802553372880683, 1886751662102221837, ...,
        967298350074953235, 1580429568371884474,   65466946037129166])

### Agregamos la columna 'target' que va a contener:

0: si la impresion fue clickeada.

1: caso contrario.

In [16]:
def get_target(imp_id):
    if imp_id in clicks_id:
        return 0
    else:
        return 1
impressions['target'] = impressions['id'].apply(lambda imp_id: get_target(imp_id))

In [17]:
impressions['target'].value_counts()

1    3488301
0     145809
Name: target, dtype: int64

Como era de esperarse el dataset se encuentra desbalanceado ya que en la mayoria de los casos las impresiones no provocan que el usuario realice un click.

### Dividimos los datos en train - validation - test

In [18]:
train, test = train_test_split(impressions, test_size= 0.1)
train, val = train_test_split(train, test_size=0.1)

In [19]:
print(train.shape)
print(val.shape)
print(test.shape)

(2943629, 27)
(327070, 27)
(363411, 27)


In [20]:
print(train['target'].value_counts())
print(val['target'].value_counts())
print(test['target'].value_counts())

1    2825654
0     117975
Name: target, dtype: int64
1    313840
0     13230
Name: target, dtype: int64
1    348807
0     14604
Name: target, dtype: int64


Los datos están relativamente bien distribuidos en el split

### Decidamos que features van a ser utilizados en nuestros modelo

In [21]:
train.columns.values

array(['id', 'imp_time', 'r_bidfloor', 'r_age', 'r_height', 'r_width',
       'categorical_0', 'categorical_1', 'categorical_2', 'categorical_3',
       'categorical_4', 'categorical_5', 'categorical_6', 'categorical_7',
       'categorical_8', 'categorical_9', 'categorical_10',
       'categorical_11', 'categorical_12', 'categorical_13',
       'categorical_14', 'categorical_15', 'categorical_16',
       'categorical_17', 'categorical_18', 'categorical_19', 'target'],
      dtype=object)

Elegimos quedarnos solo con los atributos 'r_bidfloor', 'r_age', 'r_height' y 'r_width'

In [22]:
train.columns.values[2:6]

array(['r_bidfloor', 'r_age', 'r_height', 'r_width'], dtype=object)

In [23]:
X_train = train.filter(items=train.columns.values[2:6])
y_train = train.filter(items=['target'])
X_val = val.filter(items=val.columns.values[2:6])
y_val = val.filter(items=['target'])
X_test = test.filter(items=test.columns.values[2:6])
y_test = test.filter(items=['target'])

In [24]:
X_train.head()

,r_bidfloor,r_age,r_height,r_width
824527,0.170000,NaN,NaN,NaN
2539004,0.020000,NaN,320.0,480.0
2712070,0.040000,NaN,NaN,NaN
2558764,0.219178,NaN,320.0,480.0
1530295,0.020000,NaN,250.0,300.0


Complamos los valores nan con 0s

In [25]:
X_train = X_train.fillna(value=0)
X_val = X_val.fillna(value=0)
X_test = X_test.fillna(value=0)

### Si bien faltaria visualizar un poco los datos (posiblemente luego de aplicar PCA) por cuestiones de tiempo pasemos a aplicar directamente algun modelo.

In [27]:
model = LogisticRegression(random_state=0)

In [39]:
model.fit(np.array(X_train), y_train.values.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [49]:
y_pred = model.predict(X_val)
print('Validation f1_score', f1_score(y_val, y_pred))
print('Validation accuracy_score', accuracy_score(y_val, y_pred))
y_pred = model.predict(X_test)
print('Test f1_score', f1_score(y_test, y_pred))
print('Test accuracy_score', accuracy_score(y_test, y_pred))

Validation f1_score 0.9793190612907252
Validation accuracy_score 0.9594765646497692
Test f1_score 0.9794434568691711
Test accuracy_score 0.9597150333919446


Conclusiones hasta el momento:

Se obtienen buenos resultados simplemente utilizando el modelo LogisticRegression sin realizar ajuste de hiperparámetros o incluso una mejor ingenieria de features, lo cual nos da un baseline muy prometedor.

Al ser un dataset desbalanceado a nivel de clases en particular es bueno prestarle atención a la métrica f1_score en lugar de accuracy. Se podría también trabajar con técnicas de upsampling o downsampling sobre los datos para balancear los datos.

También se podrían utilizar otros modelos como SVM o RandomForest y ver que performance se obtiene. En caso de que ajustando hiperparametros o cambiando a otro modelo lineal no se obtengan mejores resultados siempre se puede intentar con modelos no lineales más complejos como son las redes neuronales que pueden ajustar a priori mejor los datos cuando los mismos no son linealmente separables.